### Crypto Fear & Greed Index


In [1]:
# # import sys
# # sys.path.append('./src')

# from crypto_dashboard.fear_greed_index import get_index, index_data_to_pandas

In [7]:
import requests
import logging
import pandas as pd
import yfinance as yf
import warnings

from typing import Union

# configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

warnings.filterwarnings("ignore")

In [ ]:
URL = "https://api.alternative.me/fng/?limit={limit}&format={format}"


def get_index(
    url: str,
    timeout: int = 10,
    limit: int = 10,
    format: str = "json"
) -> Union[dict, None]:
    """
    Fetches the Fear & Greed Index data from the specified URL.

    Parameters:
    - url (str): 
        The API endpoint URL with placeholders for limit and format.
    - timeout (int): 
        The timeout for the HTTP request in seconds.
    - limit (int): 
        The number of data points to retrieve.
    - format (str):
        The response format, either 'json' or 'csv'.

    Returns:
    - dict or None: The JSON response as a dictionary if successful, otherwise None.
    """
    url = url.format(limit=limit, format=format)
    try:
        response = requests.get(url, timeout=timeout)
        data = response.json()['data']

        if len(data) == 0:
            logger.info("No data received from Fear & Greed API.")
            return None
        return data
    
    except Exception as e:
        logger.info("Error getting Fear & Greed data: %s", e)
        return None
    
# test
index_data = get_index(URL, limit=10, format="json")
len(index_data)

10

In [8]:
def index_data_to_pandas(index_data: dict) -> Union[pd.DataFrame, None]:
    """
    Converts the Fear & Greed Index data into a pd.DataFrame.

    Parameters:
    - index_data (dict): 
        The Fear & Greed Index data.

    Returns:
    - pd.DataFrame | None: 
        A DataFrame containing the index data, or None if input is invalid.
    """
    # delete later
    columns_to_drop = [
        'timestamp',
        'time_until_update'
    ]

    try:
        df = pd.DataFrame(index_data)
        # preprocess data
        df['value'] = pd.to_numeric(df['value'], errors='coerce')
        df['value_classification'] = df['value_classification'].astype('str')
        df['date'] = pd.to_datetime(df['timestamp'], unit='s')
        df.drop(columns=columns_to_drop, inplace=True)
        return df.sort_values(by='date').reset_index(drop=True)
    except Exception as e:
        logger.info("Error processing data into DataFrame: %s", e)
        return None
    
# test
index_data_df = index_data_to_pandas(index_data=index_data)
index_data_df

INFO:__main__:Error processing data into DataFrame: 'timestamp'


In [9]:
def get_index_data(url: str, limit: int = 10, format: str = "json") -> Union[pd.DataFrame, None]:
    """
    """
    index_data = get_index(url=url, limit=limit, format=format)
    return index_data_to_pandas(index_data=index_data)

In [10]:
index_data = get_index_data(URL, limit=10, format="json")
index_data

,value,value_classification,date
0,38,Fear,2025-10-13
1,38,Fear,2025-10-14
2,34,Fear,2025-10-15
3,28,Fear,2025-10-16
4,22,Extreme Fear,2025-10-17
5,23,Extreme Fear,2025-10-18
6,29,Fear,2025-10-19
7,29,Fear,2025-10-20
8,34,Fear,2025-10-21
9,25,Extreme Fear,2025-10-22


### Подключение к БД | Отправка данных в БД

In [ ]:
from sqlalchemy import create_engine, Table, Column, Integer, String, Date, MetaData

# подключение к Postgres (логин/пароль/БД из docker-compose)
user = "admin"
password = "admin"
db_name = "mydb"
host = "localhost"
port = 5432

engine = create_engine(f"postgresql+psycopg2://{user}:{password}@{host}:{port}/{db_name}")

In [ ]:
# отправка данных в БД
index_data.to_sql("fear_greed_index", engine, if_exists="append", index=False)

10

In [5]:
import yfinance as yf

def get_stockmarket(ticker_name: str = "^GSPC", period: str = "1mo") -> Union[pd.DataFrame, None]:
    """
    Fetches historical stock market data for the given ticker.

    Parameters:
    - ticker_name (str): 
        The stock ticker symbol. Default is "^GSPC"
    - period (str): 
        The period for which to fetch data. Default is "1mo".

    Returns:
    - pd.DataFrame | None: A DataFrame containing the historical stock data, or None
    """
    try:
        data = yf.Ticker(ticker_name).history(period=period)

        if data.empty:
            logger.info("No stock market data found for ticker: %s", ticker_name)
            return None
        return data
    except Exception as e:
        logger.error("Error getting stock market data: %s", e)
        return None
    
# test
data = get_stockmarket(ticker_name="^GSPC", period="1mo")
data.head()

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2025-09-04 00:00:00-04:00,6456.600098,6502.540039,6445.979980,6502.080078,4670770000,0.0,0.0
2025-09-05 00:00:00-04:00,6529.080078,6532.649902,6443.979980,6481.500000,5066120000,0.0,0.0
2025-09-08 00:00:00-04:00,6498.089844,6508.669922,6483.290039,6495.149902,5211500000,0.0,0.0
2025-09-09 00:00:00-04:00,6503.330078,6518.229980,6483.080078,6512.609863,4798350000,0.0,0.0
2025-09-10 00:00:00-04:00,6550.290039,6555.970215,6516.339844,6532.040039,5253010000,0.0,0.0


In [116]:
def preprocess_stockmarket_data(data: pd.DataFrame) -> pd.DataFrame:
    """
    Preprocesses the stock market data DataFrame.

    Parameters:
    - data (pd.DataFrame): 
        The raw stock market data.

    Returns:
    - pd.DataFrame: The preprocessed stock market data.
    """
    columns_to_drop = [
        'Date'
    ]

    # fix date
    data.reset_index(inplace=True)
    data['date'] = data['Date'].dt.date
    data.sort_values(by='date', ascending=False, inplace=True)

    # # add estimate column TODO: fix logic
    # data['estimate'] = data['Close'].apply(lambda x: "Rising" if x[0] > x[-1] else "Falling")

    # drop irrelevant columns
    data.drop(columns=columns_to_drop, inplace=True)
    return data

# test
data = preprocess_stockmarket_data(data=data)
data

,level_0,index,Open,High,Low,Close,Volume,Dividends,Stock Splits,date
0,22,22,6722.140137,6750.870117,6707.430176,6708.549805,1574924000,0.0,0.0,2025-10-03
1,21,21,6731.310059,6731.939941,6693.229980,6715.350098,5416130000,0.0,0.0,2025-10-02
2,20,20,6664.919922,6718.479980,6656.200195,6711.200195,6037950000,0.0,0.0,2025-10-01
3,19,19,6656.189941,6691.250000,6641.000000,6688.459961,6057210000,0.0,0.0,2025-09-30
4,18,18,6661.580078,6677.310059,6644.490234,6661.209961,5358760000,0.0,0.0,2025-09-29
5,17,17,6615.379883,6648.970215,6604.430176,6643.700195,5103110000,0.0,0.0,2025-09-26
6,16,16,6608.189941,6619.000000,6569.220215,6604.720215,5874670000,0.0,0.0,2025-09-25
7,15,15,6669.790039,6672.660156,6621.759766,6637.970215,5459180000,0.0,0.0,2025-09-24
8,14,14,6692.439941,6699.520020,6645.580078,6656.919922,5633620000,0.0,0.0,2025-09-23
9,13,13,6654.279785,6698.879883,6648.069824,6693.750000,5642620000,0.0,0.0,2025-09-22


In [110]:
data.reset_index(inplace=True)
data.head()

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2025-09-03 00:00:00-04:00,6445.819824,6453.669922,6416.169922,6448.259766,4465360000,0.0,0.0
1,2025-09-04 00:00:00-04:00,6456.600098,6502.540039,6445.979980,6502.080078,4670770000,0.0,0.0
2,2025-09-05 00:00:00-04:00,6529.080078,6532.649902,6443.979980,6481.500000,5066120000,0.0,0.0
3,2025-09-08 00:00:00-04:00,6498.089844,6508.669922,6483.290039,6495.149902,5211500000,0.0,0.0
4,2025-09-09 00:00:00-04:00,6503.330078,6518.229980,6483.080078,6512.609863,4798350000,0.0,0.0


In [112]:
data['date'] = data['Date'].dt.date
data

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,date
0,2025-09-03 00:00:00-04:00,6445.819824,6453.669922,6416.169922,6448.259766,4465360000,0.0,0.0,2025-09-03
1,2025-09-04 00:00:00-04:00,6456.600098,6502.540039,6445.979980,6502.080078,4670770000,0.0,0.0,2025-09-04
2,2025-09-05 00:00:00-04:00,6529.080078,6532.649902,6443.979980,6481.500000,5066120000,0.0,0.0,2025-09-05
3,2025-09-08 00:00:00-04:00,6498.089844,6508.669922,6483.290039,6495.149902,5211500000,0.0,0.0,2025-09-08
4,2025-09-09 00:00:00-04:00,6503.330078,6518.229980,6483.080078,6512.609863,4798350000,0.0,0.0,2025-09-09
5,2025-09-10 00:00:00-04:00,6550.290039,6555.970215,6516.339844,6532.040039,5253010000,0.0,0.0,2025-09-10
6,2025-09-11 00:00:00-04:00,6554.410156,6592.890137,6545.799805,6587.470215,5426460000,0.0,0.0,2025-09-11
7,2025-09-12 00:00:00-04:00,6590.660156,6600.209961,6579.490234,6584.290039,4641640000,0.0,0.0,2025-09-12
8,2025-09-15 00:00:00-04:00,6603.490234,6619.620117,6602.069824,6615.279785,5045020000,0.0,0.0,2025-09-15
9,2025-09-16 00:00:00-04:00,6624.129883,6626.990234,6600.109863,6606.759766,5359510000,0.0,0.0,2025-09-16


In [ ]:


def figure_stockmarket(data):
    try:
        estimate = 0
        df = data.reset_index()[['Date', 'Close']].copy()
        df = df.rename(columns={'Date': 'date', 'Close': 'stockmarket_value'})
        df['date'] = df['date'].dt.date
        estimate = "Rising" if data.iloc[-1, 1] > data.iloc[0, 1] else "Falling"
        date = datetime.now().date()
        stockmarket = pd.DataFrame({'date': [date], 'stockmarket': [estimate]})
        print(f"date is type: {type(stockmarket['date'].iloc[0])}")
        print(f"data is type: {type(stockmarket)}")
        return stockmarket
    except Exception as e:
        print(f"Error cleaning stock market data: {e}")
        return None

data is type: <class 'pandas.core.frame.DataFrame'>


In [106]:
data

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2025-09-03 00:00:00-04:00,6445.819824,6453.669922,6416.169922,6448.259766,4465360000,0.0,0.0
2025-09-04 00:00:00-04:00,6456.600098,6502.540039,6445.979980,6502.080078,4670770000,0.0,0.0
2025-09-05 00:00:00-04:00,6529.080078,6532.649902,6443.979980,6481.500000,5066120000,0.0,0.0
2025-09-08 00:00:00-04:00,6498.089844,6508.669922,6483.290039,6495.149902,5211500000,0.0,0.0
2025-09-09 00:00:00-04:00,6503.330078,6518.229980,6483.080078,6512.609863,4798350000,0.0,0.0
2025-09-10 00:00:00-04:00,6550.290039,6555.970215,6516.339844,6532.040039,5253010000,0.0,0.0
2025-09-11 00:00:00-04:00,6554.410156,6592.890137,6545.799805,6587.470215,5426460000,0.0,0.0
2025-09-12 00:00:00-04:00,6590.660156,6600.209961,6579.490234,6584.290039,4641640000,0.0,0.0
2025-09-15 00:00:00-04:00,6603.490234,6619.620117,6602.069824,6615.279785,5045020000,0.0,0.0
